# Script to Compute Indices/Variables

This script demonstrates the computation of indices/variables necessary for this analysis.

**Following steps are included in this script:**

1. Load monthly netCDF files
2. Load daily netCDF files
3. Compute WUE
4. Compute VPD
5. Compute Evaporation
6. Compute RX5day
7. & 8. Save the processed data to a specified path.

In [ ]:
# ========== Import Required Libraries ==========
import sys
import dask
from dask.diagnostics import ProgressBar
import os
import xarray as xr

In [ ]:
# ========== Configure Paths ==========
# Define the full path to the directories containing utility scripts and configurations
data_handling_dir = '../../src/data_handling'
config_file = '../../src'

# Add directories to sys.path for importing custom modules
sys.path.append(data_handling_dir)
sys.path.append(config_file)

# Import custom utility functions and configurations
import load_data as ld
import process_data as pro_dat
import save_data_as_nc as sd
from config import DATA_DIR, DEFAULT_EXPERIMENT, DEFAULT_MODEL

### Step 1: Load monthly netCDF files

In [ ]:
# Step 1.1: Define the datasets
data_state = 'processed' # Using already preprocessing data
experiments = [DEFAULT_EXPERIMENT] # You can load multiple experiments here with [experiment_id_1, experiment_id_2, ...] 
models = [DEFAULT_MODEL] # You can load multiple models here with [Model_name_1, Model_name_2 ...]
variables=['gpp', 'tran', # Necessary for WUE
           'ps', 'huss', 'tas', # Necessary for VPD
           'evspsbl', 'tran' # Necessary for Evaporation
]

In [ ]:
# Step 1.2: Load the datasets
print("Loading datasets...")
with ProgressBar():
    ds_dict_month = dask.compute(
        ld.load_multiple_models_and_experiments(
            DATA_DIR, data_state, 'CMIP6', experiments, 'month', models, variables
        )
    )[0]

### Step 2: Load daily netCDF files

In [ ]:
# Step 2.1: Define the datasets
data_state = 'processed' # Using already preprocessing data
experiments = [DEFAULT_EXPERIMENT] # You can load multiple experiments here with [experiment_id_1, experiment_id_2, ...] 
models = [DEFAULT_MODEL] # You can load multiple models here with [Model_name_1, Model_name_2 ...]
variables=['pr' # Necessary for RX5day
]

In [ ]:
# Step 2.2: Load the datasets
print("Loading datasets...")
with ProgressBar():
    ds_dict_day = dask.compute(
        ld.load_multiple_models_and_experiments(
            DATA_DIR, data_state, 'CMIP6', experiments, 'day', models, variables
        )
    )[0]

### Step 2: Compute WUE

In [ ]:
ds_dict_month[DEFAULT_EXPERIMENT] = pro_dat.compute_wue(ds_dict_month[DEFAULT_EXPERIMENT])

### Step 3: Compute VPD

In [ ]:
ds_dict_month[DEFAULT_EXPERIMENT] = pro_dat.compute_vpd(ds_dict_month[DEFAULT_EXPERIMENT])

### Step 4: Compute Evaporation

In [ ]:
ds_dict_month[DEFAULT_EXPERIMENT] = pro_dat.compute_evapo(ds_dict_month[DEFAULT_EXPERIMENT])

### Step 5: Compute RX5day

In [ ]:
ds_dict_day[DEFAULT_EXPERIMENT] = pro_dat.compute_RX5day(ds_dict_day[DEFAULT_EXPERIMENT])

### Step 6: Remove base variables

In [ ]:
# List of variables to remove
variables_to_remove = ['gpp', 'tran', 'ps', 'huss', 'tas', 'evspsbl']

# Remove variables from the dataset dictionary
ds_dict_month[DEFAULT_EXPERIMENT] = pro_dat.remove_variables(ds_dict_month[DEFAULT_EXPERIMENT], variables_to_remove)

### Step 7: Define Output File Path

In [ ]:
# Construct the output file path
data_path_day = f"processed/CMIP6/{DEFAULT_EXPERIMENT}/day/"
file_path_day = os.path.join(DATA_DIR, data_path_day)
print(f"Saving daily files to: {file_path_day}")

In [ ]:
# Construct the output file path
data_path_month = f"processed/CMIP6/{DEFAULT_EXPERIMENT}/month/"
file_path_month = os.path.join(DATA_DIR, data_path_month)
print(f"Saving monthly files to: {file_path_month}")

### Step 8: Save Data

In [ ]:
# Save the processed datasets and remove any existing files at the target path
sd.save_files(ds_dict_day[DEFAULT_EXPERIMENT], file_path_day)
sd.save_files(ds_dict_month[DEFAULT_EXPERIMENT], file_path_month)